ETL fPagamentos


In [3]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
from dotenv import load_dotenv
load_dotenv(dotenv_path=r"C:\Users\livsa\DM_Financeiro\Financial-Data-Engineering-Flow\config\.env")
import os


In [13]:
df = pd.read_csv(r'C:\Users\livsa\Downloads\BaseDados-20251130T002854Z-1-001\BaseDados\TitulosFinanceiros\LancamentosFinanceiros.csv',sep=";")
df.head(5)

C:\Users\livsa\AppData\Local\Temp\ipykernel_3892\654574662.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\livsa\Downloads\BaseDados-20251130T002854Z-1-001\BaseDados\TitulosFinanceiros\LancamentosFinanceiros.csv',sep=";")


,Data de Emissao,Data de Vencimento,Data de Pagamento,CodFilial,CodEmpresa,CodTipoDespesa,CodContaContabil,CodBanco,CodCategoria,Origem,Tipo Movimento,Status,CodMovimento,Parcela,Programacao,Valor Original,Valor Juros,Valor Multa,Valor Desconto,Valor Pago
0,16/08/2018,15/10/2018,15/10/2018,1001,4388,2008,29,5,2,TITULO A PAGAR,Saída,REALIZADO,55226,1,1,2560,0,0,0,2560
1,16/08/2018,15/10/2018,15/10/2018,1001,4388,2008,29,5,2,TITULO A PAGAR,Saída,REALIZADO,55225,1,1,2560,0,0,0,2560
2,16/08/2018,15/10/2018,15/10/2018,1001,4388,2008,29,5,2,TITULO A PAGAR,Saída,REALIZADO,55223,1,1,2560,0,0,0,2560
3,17/08/2018,16/10/2018,16/10/2018,1001,4388,2008,29,5,2,TITULO A PAGAR,Saída,REALIZADO,55274,1,1,2560,0,0,0,2560
4,22/08/2018,21/10/2018,22/10/2018,1001,4388,2008,29,5,2,TITULO A PAGAR,Saída,REALIZADO,55483,1,1,2560,0,0,0,2560


In [3]:
df.columns

Index(['Data de Emissao', 'Data de Vencimento', 'Data de Pagamento',
       'CodFilial', 'CodEmpresa', 'CodTipoDespesa', 'CodContaContabil',
       'CodBanco', 'CodCategoria', 'Origem', 'Tipo Movimento', 'Status',
       'CodMovimento', 'Parcela', 'Programacao', 'Valor Original',
       'Valor Juros', 'Valor Multa', 'Valor Desconto', 'Valor Pago'],
      dtype='object')

In [ ]:
df = df[df["Tipo Movimento"] == "Entrada"]



In [16]:
colunas_excluir = ["Origem", "Tipo Movimento"]
df = df.drop(columns=colunas_excluir)

In [7]:
for coluna in df.columns:
    print(f"Coluna: {coluna}, Tipo de dados: {df[coluna].dtypes}")


Coluna: Data de Emissao, Tipo de dados: datetime64[ns]
Coluna: Data de Vencimento, Tipo de dados: datetime64[ns]
Coluna: Data de Pagamento, Tipo de dados: datetime64[ns]
Coluna: CodFilial, Tipo de dados: int64
Coluna: CodEmpresa, Tipo de dados: int64
Coluna: CodTipoDespesa, Tipo de dados: int64
Coluna: CodContaContabil, Tipo de dados: int64
Coluna: CodBanco, Tipo de dados: int64
Coluna: CodCategoria, Tipo de dados: int64
Coluna: Origem, Tipo de dados: object
Coluna: Tipo Movimento, Tipo de dados: object
Coluna: Status, Tipo de dados: object
Coluna: CodMovimento, Tipo de dados: int64
Coluna: Parcela, Tipo de dados: int64
Coluna: Programacao, Tipo de dados: int64
Coluna: Valor Original, Tipo de dados: object
Coluna: Valor Juros, Tipo de dados: object
Coluna: Valor Multa, Tipo de dados: object
Coluna: Valor Desconto, Tipo de dados: object
Coluna: Valor Pago, Tipo de dados: object


In [31]:
colunas_data = ['Data de Emissao', 'Data de Vencimento', 'Data de Pagamento']

# Converte todas as colunas da lista para datetime
df[colunas_data] = df[colunas_data].apply(pd.to_datetime, errors='coerce')

for col in colunas_data:
    df[col] = pd.to_datetime(df[col], errors="coerce")  # garante datetime
    df[col] = df[col].astype("object")                 
    df[col] = df[col].where(df[col].notnull(), None)    # NaT → None 


In [ ]:
load_dotenv()

# ----------- PostgreSQL ----------
pg_user = os.getenv("POSTGRES_USER")
pg_password = os.getenv("POSTGRES_PASSWORD")
pg_host = os.getenv("POSTGRES_HOST")
pg_port = os.getenv("POSTGRES_PORT")
pg_database = os.getenv("POSTGRES_DB")

conexao = psycopg2.connect(
    dbname=pg_database,
    user=pg_user,
    password=pg_password,
    host=pg_host,
    port=pg_port
)
cursor = conexao.cursor()

# Limpa a tabela
cursor.execute("TRUNCATE TABLE public.fPagamentos;")

# ---------- CONVERSÃO DOS VALORES NUMÉRICOS ----------
colunas_valores = [
    "Valor Original",
    "Valor Juros",
    "Valor Multa",
    "Valor Desconto",
    "Valor Pago"
]

for col in colunas_valores:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(".", "", regex=False)   # remove pontos de milhar
        .str.replace(",", ".", regex=False)  # troca vírgula por ponto
        .astype(float)                       # float real
    )

# ---------- MONTANDO A LISTA DE VALORES JÁ CORRIGIDOS ----------
valores = [
    (
        row["Data de Emissao"],
        row["Data de Vencimento"],
        row["Data de Pagamento"],
        row["CodFilial"],
        row["CodEmpresa"],
        row["CodTipoDespesa"],
        row["CodContaContabil"],
        row["CodBanco"],
        row["CodCategoria"],
        row["Origem"],
        row["Tipo Movimento"],
        row["Status"],
        row["CodMovimento"],
        row["Parcela"],
        row["Programacao"],
        row["Valor Original"],
        row["Valor Juros"],
        row["Valor Multa"],
        row["Valor Desconto"],
        row["Valor Pago"]
    )
    for index, row in df.iterrows()
]

# ---------- SQL DE INSERT ----------
sql_insert = """
INSERT INTO public.fPagamentos (
    Data_de_Emissao,
    Data_de_Vencimento,
    Data_de_Pagamento,
    CodFilial,
    CodEmpresa,
    CodTipoDespesa,
    CodContaContabil,
    CodBanco,
    CodCategoria,
    Origem,
    Tipo_Movimento,
    Status,
    CodMovimento,
    Parcela,
    Programacao,
    Valor_Original,
    Valor_Juros,
    Valor_Multa,a
    Valor_Desconto,
    Valor_Pago
) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
"""

# ---------- EXECUÇÃO OTIMIZADA ----------
execute_batch(cursor, sql_insert, valores, page_size=5000)

conexao.commit()
cursor.close()
conexao.close()

print("Carga realizada com sucesso!")

Carga realizada com sucesso!


ETL dPagamentos


In [1]:
df2= pd.read_excel(r"C:\Users\livsa\DM_Financeiro\Financial-Data-Engineering-Flow\data\PlanoContas.xlsx",sheet_name="PlanoContas")
df2.head(5)


NameError: name 'pd' is not defined

In [35]:
df2.columns

Index(['CodGrupoDRE', 'CodContaContabil', 'ContaContabil', 'Classificacao',
       'Nivel1DRE', 'Nivel2DRE'],
      dtype='object')

In [42]:
colunas_nivel= ['Nivel1DRE', 'Nivel2DRE']
df2[colunas_nivel] = df2[colunas_nivel].fillna("NÃO SE APLICA")


ETL mascaraDRE


In [4]:
df3= pd.read_excel(r"C:\Users\livsa\DM_Financeiro\Financial-Data-Engineering-Flow\data\PlanoContas.xlsx",sheet_name="MascaraDRE")
df3.head(5)

,CodGrupoDRE,ContaGerencial,Subtotal
0,1,RECEITA OPERACIONAL BRUTA,0
1,2,Deduções da receita,0
2,3,RECEITA OPERACIONAL LÍQUIDA,1
3,4,Custo da mercadoria vendida (CMV),0
4,5,Lucro Bruto,1


In [9]:
df3.columns

Index(['CodGrupoDRE', 'ContaGerencial', 'Subtotal'], dtype='object')

In [29]:
df4 = pd.read_excel(r"C:\Users\livsa\DM_Financeiro\Financial-Data-Engineering-Flow\data\Cadastros.xlsx","Filial",header=2)

df4.head(5)

,CodFilial,NomeFilial,TipoFilial
0,1001,MATRIZ,MATRIZ
1,1002,FILIAL 2,FILIAL
2,1003,FILIAL 3,FILIAL
3,1004,FILIAL 4,FILIAL


In [32]:
df4 = df4.rename(columns={'NomeFilial': 'Nome Fantasia'})
df4.columns


Index(['CodFilial', 'Nome Fantasia', 'TipoFilial'], dtype='object')

In [35]:
# Diretório dos arquivos Excel
diretorio = (r"C:\Users\livsa\DM_Financeiro\Financial-Data-Engineering-Flow\data\LancamentosContabilidade")

# Lista para armazenar os dados dos arquivos
dados = []

# Iterar sobre os arquivos Excel no diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(diretorio, arquivo)
        df = pd.read_csv(caminho_arquivo,sep=";")
        dados.append(df)

# Consolidar os dados em um único DataFrame
df5 = pd.concat(dados)
df5.head()


,CodFilial,DataMovimento,CodConta,CodCentroCusto,StatusLançamento,TipoLancamento,Lote,LoteItem,Valor
0,1001,02/01/2018,531,220,INTEGRADO,D,277810,8,"1785,6832"
1,1001,02/01/2018,531,220,INTEGRADO,D,277810,9,"3482,0864"
2,1001,02/01/2018,531,220,INTEGRADO,D,277810,11,"3571,3728"
3,1001,02/01/2018,302,200,INTEGRADO,D,277810,12,"3029,7984"
4,1001,02/01/2018,302,200,INTEGRADO,D,277810,6,"2844,0448"


In [43]:
df5 = df5.rename(columns={'DataMovimento': 'Data do Movimento','TipoLancamento':'Tipo do Lancamento'})
df5.columns

Index(['CodFilial', 'Data do Movimento', 'CodConta', 'CodCentroCusto',
       'StatusLançamento', 'Tipo do Lancamento', 'Lote', 'LoteItem', 'Valor'],
      dtype='object')

In [42]:
for coluna in df5.columns:
    print(f"Coluna: {coluna}, Tipo de dados: {df5[coluna].dtypes}")


Coluna: CodFilial, Tipo de dados: int64
Coluna: Data do Movimento, Tipo de dados: datetime64[ns]
Coluna: CodConta, Tipo de dados: int64
Coluna: CodCentroCusto, Tipo de dados: int64
Coluna: StatusLançamento, Tipo de dados: object
Coluna: Tipo do Lancamento, Tipo de dados: object
Coluna: Lote, Tipo de dados: int64
Coluna: LoteItem, Tipo de dados: int64
Coluna: Valor, Tipo de dados: object


In [41]:
cols_data = ['Data do Movimento']

for col in cols_data:
    # Converte para datetime, usando dayfirst=True
    df5[col] = pd.to_datetime(df5[col], dayfirst=True, errors='coerce')
    
    # Converte NaT para None (compatível com PostgreSQL)
    df5[col] = df5[col].where(df5[col].notnull(), None)
